<a href="https://colab.research.google.com/github/atomicai/BERTopic/blob/master/vtt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install faster_whisper ffmpeg loguru bitsandbytes googletrans==3.1.0a0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from subprocess import run
from faster_whisper import WhisperModel
import json
import tempfile
import os
import ffmpeg
from pathlib import Path
from zipfile import ZipFile
from loguru import logger
import stat
import uuid
import subprocess
import torch
import bitsandbytes
import scipy

from googletrans import Translator
import re
import subprocess
import datetime

#ZipFile("ffmpeg.zip").extractall()
#st = os.stat("ffmpeg")
#os.chmod("ffmpeg", st.st_mode | stat.S_IEXEC)


# with open("google_lang_codes.json", "r") as f:
#     google_lang_codes = json.load(f)

google_lang_codes = dict(
    Russian="ru",
    English="en"
)

translator = Translator()
whisper_model = WhisperModel("large-v2", device="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!ls ./drive/MyDrive/YAPPI

top-100_transcript_response.csv  top-160_transcript_response.csv  top-40_transcript_response.csv
top-10_transcript_response.csv	 top-170_transcript_response.csv  top-50_transcript_response.csv
top-110_transcript_response.csv  top-180_transcript_response.csv  top-60_transcript_response.csv
top-120_transcript_response.csv  top-190_transcript_response.csv  top-70_transcript_response.csv
top-130_transcript_response.csv  top-200_transcript_response.csv  top-80_transcript_response.csv
top-140_transcript_response.csv  top-20_transcript_response.csv   top-90_transcript_response.csv
top-150_transcript_response.csv  top-30_transcript_response.csv


In [10]:
import polars as pl
import pandas as pd

In [11]:
pl_docs = pl.read_csv("./drive/MyDrive/yappi_hackaton_2024_400k.csv")

In [12]:
pl_docs = pl_docs.filter(pl.col("description").is_not_null())

In [13]:
pl_docs.head()

link,description
str,str
"""https://cdn-st…","""#нарезкистримо…"
"""https://cdn-st…","""🤫НЕ ВВОДИ ЭТУ …"
"""https://cdn-st…","""#boobs , #крас…"
"""https://cdn-st…","""#diy #постанов…"
"""https://cdn-st…","""#образ #lookbo…"


In [15]:
words_to_consider = ["егэ2023"]

In [17]:
pl_docs = pl_docs.filter(pl.col("description").str.contains_any(words_to_consider))

In [19]:
from tqdm.autonotebook import tqdm
from more_itertools import chunked
import asyncio as asio
import smart_open as so

In [20]:
docs = pl_docs.select("link").to_series().to_list()

In [21]:
async def parallel_download(url):
  # Download video via url.
  # 1) Создаем уникальный .wav файл
  # 2) Качаем
  # 3) Возвращаем
  fname = f"{uuid.uuid4()}.wav"
  proc = await asio.create_subprocess_exec(
   'ffmpeg','-i', url, fname,
   stdout=asio.subprocess.PIPE,
   stderr=asio.subprocess.PIPE)
  stdout, stderr = await proc.communicate()
  return fname

In [22]:
import smart_open as so

def stream_uri(uri_in, uri_out, chunk_size=1 << 18):  # 256kB chunks
    """Write from uri_in to uri_out with minimal memory footprint."""
    with so.open(uri_in, "rb") as fin, so.open(uri_out, "wb") as fout:
        while chunk := fin.read(chunk_size):
            fout.write(chunk)
    return uri_out

In [24]:
async def ignite_video_processing(fpath: Path, codes_mapping, lang_to_translate = "Russian", translate_video: bool = True):
    cur_uuid = str(Path(fpath).stem)
    cur_path = Path(os.getcwd())
    transcript_file = str(Path(fpath).stem) + ".srt"
    print(transcript_file)
    # Transcription with Whisper.
    target_language_code = codes_mapping.get(lang_to_translate, "ru")
    logger.info(f"Using target lagnuage code {lang_to_translate}")
    segments, _ = whisper_model.transcribe(fpath, beam_size=5)
    segments = list(segments)

    with open(transcript_file, "w+", encoding="utf-8") as f:
        counter = 1
        for segment in segments:
            start_hours = int(segment.start // 3600)
            start_minutes = int((segment.start % 3600) // 60)
            start_seconds = int(segment.start % 60)
            start_milliseconds = int((segment.start - int(segment.start)) * 1000)

            end_hours = int(segment.end // 3600)
            end_minutes = int((segment.end % 3600) // 60)
            end_seconds = int(segment.end % 60)
            end_milliseconds = int((segment.end - int(segment.end)) * 1000)

            formatted_start = f"{start_hours:02d}:{start_minutes:02d}:{start_seconds:02d},{start_milliseconds:03d}"
            formatted_end = f"{end_hours:02d}:{end_minutes:02d}:{end_seconds:02d},{end_milliseconds:03d}"

            f.write(f"{counter}\n")
            f.write(f"{formatted_start} --> {formatted_end}\n")
            f.write(f"{segment.text}\n\n")
            counter += 1

        # Check if translation is needed
        if translate_video:
            # Translating the SRT from Whisper with Google Translate.
            translated_lines = []
            f.seek(0)  # Move the file pointer to the beginning of the file.
            for line in f:
                if line.strip().isnumeric() or "-->" in line:
                    translated_lines.append(line)
                elif line.strip() != "":
                    translated_text = translator.translate(line.strip(), dest=target_language_code).text
                    translated_lines.append(translated_text + "\n")
                else:
                    translated_lines.append("\n")

            f.seek(0)  # Move the file pointer to the beginning of the file and truncate it.
            f.truncate()
            f.writelines(translated_lines)  # Write the translated lines back into the original file.
    # Debugging: Validate FFmpeg command for subtitle embedding
    print("Validating FFmpeg command for subtitle embedding...")
    print(f"Translated SRT file: {transcript_file}")
    os.unlink(str(fpath))
    os.unlink(str(transcript_file))
    return translated_lines

In [29]:
offset = 0
batch_size = 16

In [26]:
probs = []

In [27]:
snapshot_name = "_".join(words_to_consider)

In [ ]:
pl_res = pl.DataFrame(schema={"url": str, "ans": str})

for i, docs_batch in tqdm(enumerate(chunked(docs[offset:], batch_size))):
    docs_paths = await asio.gather(*[parallel_download(doc) for doc in docs_batch])
    prob_paths = [dict(url=doc, idx=idx) for doc, idx in zip(docs_batch, docs_paths) if not Path(idx).exists()]
    probs.extend(prob_paths)
    docs_paths = [dpi for dpi in docs_paths if Path(dpi).exists()]
    resx_trans = await asio.gather(*[ignite_video_processing(dp, google_lang_codes) for dp in docs_paths])
    response = []
    for url, ans in zip(docs_batch, resx_trans):
      response.append(dict(
          url=url,
          ans=" ".join(ans)
      ))
    pl_response = pl.from_dicts(response)
    pl_res = pl.concat([pl_res, pl_response])
    if (i + 1) %  10 == 0:
        pl_res.write_csv(f"./drive/MyDrive/YAPPI/top-{str(i + 1)}_transcript_{snapshot_name}_response.csv") # to disk save

0it [00:00, ?it/s]

2024-06-12 11:20:12.437 | INFO     | __main__:ignite_video_processing:8 - Using target lagnuage code Russian


8c06802e-1df6-43eb-8980-ff5e92cfe712.srt


2024-06-12 11:20:15.449 | INFO     | __main__:ignite_video_processing:8 - Using target lagnuage code Russian


Validating FFmpeg command for subtitle embedding...
Translated SRT file: 8c06802e-1df6-43eb-8980-ff5e92cfe712.srt
d42f7762-2383-4c76-8c49-f68f92717f64.srt


...

Validating FFmpeg command for subtitle embedding...
Translated SRT file: 4a53142c-c930-4c9a-914b-b5bcb5c07e20.srt
835e4285-fa7a-4743-8443-c1de05ab71d1.srt


2024-06-12 11:40:39.967 | INFO     | __main__:ignite_video_processing:8 - Using target lagnuage code Russian


Validating FFmpeg command for subtitle embedding...
Translated SRT file: 835e4285-fa7a-4743-8443-c1de05ab71d1.srt
09c3fbfb-01fe-4707-9166-d97bf2621c80.srt


2024-06-12 11:40:40.721 | INFO     | __main__:ignite_video_processing:8 - Using target lagnuage code Russian


Validating FFmpeg command for subtitle embedding...
Translated SRT file: 09c3fbfb-01fe-4707-9166-d97bf2621c80.srt
13cecc98-5a43-4f9b-baee-64f99322c52d.srt


In [ ]:
!rm -rf /content/*.mp4
!rm -rf /content/*.srt

In [ ]:
pl_res.write_csv(f"./drive/MyDrive/YAPPI/top-{str(i + 1)}_transcript_response.csv") # to disk save

In [ ]:
Path(docs_batch[0])

PosixPath('https:/cdn-st.rutubelist.ru/media/b0/e9/ef285e0241139fc611318ed33071/fhd.mp4')

In [ ]:
docs_paths = await asio.gather(*[parallel_download(doc) for doc in docs_batch])

In [ ]:
Path(docs_paths[1]).stem

'f0928a98-c0f4-452a-931a-06be354a88b2'

In [ ]:
result = await ignite_video_processing(docs_paths[1], google_lang_codes)

2024-06-11 22:28:33.449 | INFO     | __main__:ignite_video_processing:8 - Using target lagnuage code Russian


f0928a98-c0f4-452a-931a-06be354a88b2.srt
Validating FFmpeg command for subtitle embedding...
Translated SRT file: f0928a98-c0f4-452a-931a-06be354a88b2.srt


In [ ]:
result

['1\n',
 '00:00:00,000 --> 00:00:02,500\n',
 'Сейчас я тебе покажу секретную команду в Роблоксе!\n',
 '\n',
 '2\n',
 '00:00:02,500 --> 00:00:06,600\n',
 'Чтобы её активировать, поставь лайк и подпишись, а также введи в чат команду ILOVEYOU!\n',
 '\n',
 '3\n',
 '00:00:06,600 --> 00:00:10,099\n',
 'Когда вы её введёте, у вас на экране появится вот такой клоун!\n',
 '\n',
 '4\n',
 '00:00:10,099 --> 00:00:15,099\n',
 'Скрипт был создан одним из создателей Роблокса и работает только в играх, где есть Роблокс Девелопер Сервис.\n',
 '\n',
 '5\n',
 '00:00:15,099 --> 00:00:18,600\n',
 'Чтобы брать скример нужно всего-то выйти из игры, но лучше не проверяй!\n',
 '\n']

In [ ]:
!pwd

/content
